In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!pip install --quiet --upgrade tensorflow_federated
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
import tensorflow as tf
# Evita di allocare tutta la memoria video a tensorflow (Chiamare solo al primo import di tf)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("No GPU found, model running on CPU")
from split_process import SplitProcess
import numpy as np
import pandas as pd
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet import MobileNet
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning) 

SEED = 1234567890
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [3]:
# Definizione dei client e dei dataset
TAKE_ONLY = None
exp_1_config = {
    'client_0': {'path':'/content/drive/MyDrive/tfrecords/train_80%-0.tfrecord', 'take_only': TAKE_ONLY},
    'client_1': {'path':'/content/drive/MyDrive/tfrecords/train_80%-1.tfrecord', 'take_only': TAKE_ONLY},
    'client_2': {'path':'/content/drive/MyDrive/tfrecords/train_80%-2.tfrecord', 'take_only': TAKE_ONLY}
}

val_config = {
    'client_0': {'path':'/content/drive/MyDrive/tfrecords/valid_20%-0.tfrecord', 'take_only': TAKE_ONLY},
    'client_1': {'path':'/content/drive/MyDrive/tfrecords/valid_20%-1.tfrecord', 'take_only': TAKE_ONLY},
    'client_2': {'path':'/content/drive/MyDrive/tfrecords/valid_20%-2.tfrecord', 'take_only': TAKE_ONLY}
}
path = '/content/drive/MyDrive/Modelli'

In [4]:
# Istanzio un nuovo processo
sp = SplitProcess(model_name='Split_MobileNet', model_architecture=MobileNet)

In [5]:
# Inizializzazione
sp.setup(exp_1_config, val_config, output_folder=path)

Creating split dataset for 3 clients
loading /content/drive/MyDrive/tfrecords/train_80%-0.tfrecord
loading /content/drive/MyDrive/tfrecords/train_80%-1.tfrecord
loading /content/drive/MyDrive/tfrecords/train_80%-2.tfrecord
loading /content/drive/MyDrive/tfrecords/valid_20%-0.tfrecord
loading /content/drive/MyDrive/tfrecords/valid_20%-1.tfrecord
loading /content/drive/MyDrive/tfrecords/valid_20%-2.tfrecord
Builiding model of client_0...
Builiding model of client_1...
Builiding model of client_2...


In [ ]:
sp.iterative_training(split_layer=60, epochs=5)

In [ ]:
sp.parallel_training(split_layer=60, epochs=5)

In [ ]:
# sp.evaluation(epochs=1, model_path='/content/drive/MyDrive/Modelli/Split/B')

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir drive/MyDrive/Modelli/Split/A/logs